# Memento Pattern

The Memento pattern captures and externalizes an object's internal state so that the object can be restored to this state later, without violating encapsulation.

## Problem

You need to save the state of an object at a particular point in time so you can restore it later, but direct access to the object's fields would violate encapsulation.

## Solution

Define three roles: the Originator (creates and uses mementos to save/restore state), the Memento (stores the internal state of the Originator), and the Caretaker (keeps track of multiple mementos but never examines their contents).

In [1]:
import time
from typing import List, Any


# Memento - stores the internal state of the Originator
class EditorMemento:
    def __init__(self, content: str):
        self._content = content
        self._date = time.strftime("%Y-%m-%d %H:%M:%S")

    @property
    def content(self) -> str:
        return self._content

    @property
    def date(self) -> str:
        return self._date

    def __str__(self) -> str:
        return f"[{self._date}] {self._content[:15]}..."

In [2]:
# Originator - creates and uses mementos to save/restore state
class Editor:
    def __init__(self):
        self._content = ""

    @property
    def content(self) -> str:
        return self._content

    @content.setter
    def content(self, content: str):
        self._content = content

    def type(self, text: str):
        self._content += text
        print(f"Editor: Added '{text}'")
        print(f"Current content: '{self._content}'")

    def save(self) -> EditorMemento:
        print("Editor: Saving state...")
        return EditorMemento(self._content)

    def restore(self, memento: EditorMemento):
        self._content = memento.content
        print(f"Editor: Restored to '{self._content}'")

In [3]:
# Caretaker - keeps track of multiple mementos
class History:
    def __init__(self, editor: Editor):
        self._editor = editor
        self._mementos: List[EditorMemento] = []
        self._current = -1

    def backup(self):
        print("\nHistory: Saving editor state...")
        # If we've undone and now make a new change,
        # discard all future states
        if self._current < len(self._mementos) - 1:
            self._mementos = self._mementos[: self._current + 1]

        self._mementos.append(self._editor.save())
        self._current = len(self._mementos) - 1

    def undo(self):
        if self._current <= 0:
            print("\nHistory: Can't undo, at earliest state")
            return

        print("\nHistory: Undoing to previous state...")
        self._current -= 1
        memento = self._mementos[self._current]
        self._editor.restore(memento)

    def redo(self):
        if self._current >= len(self._mementos) - 1:
            print("\nHistory: Can't redo, at latest state")
            return

        print("\nHistory: Redoing to next state...")
        self._current += 1
        memento = self._mementos[self._current]
        self._editor.restore(memento)

    def show_history(self):
        print("\nHistory: Here are the saved states:")
        for i, memento in enumerate(self._mementos):
            marker = "» " if i == self._current else "  "
            print(f"{marker}{i}: {memento}")

In [4]:
# Client code
if __name__ == "__main__":
    editor = Editor()
    history = History(editor)

    # Initial content
    editor.type("Hello, ")
    history.backup()

    # Add more content
    editor.type("world!")
    history.backup()

    # Add even more content
    editor.type(" How are you today?")
    history.backup()

    # Show history
    history.show_history()

    # Undo
    history.undo()
    history.show_history()

    # Undo again
    history.undo()
    history.show_history()

    # Redo
    history.redo()
    history.show_history()

    # Make a new change after undo
    editor.type(" Welcome!")
    history.backup()
    history.show_history()

    # Try to redo (should fail)
    history.redo()

Editor: Added 'Hello, '
Current content: 'Hello, '

History: Saving editor state...
Editor: Saving state...
Editor: Added 'world!'
Current content: 'Hello, world!'

History: Saving editor state...
Editor: Saving state...
Editor: Added ' How are you today?'
Current content: 'Hello, world! How are you today?'

History: Saving editor state...
Editor: Saving state...

History: Here are the saved states:
  0: [2025-04-01 22:43:54] Hello, ...
  1: [2025-04-01 22:43:54] Hello, world!...
» 2: [2025-04-01 22:43:54] Hello, world! H...

History: Undoing to previous state...
Editor: Restored to 'Hello, world!'

History: Here are the saved states:
  0: [2025-04-01 22:43:54] Hello, ...
» 1: [2025-04-01 22:43:54] Hello, world!...
  2: [2025-04-01 22:43:54] Hello, world! H...

History: Undoing to previous state...
Editor: Restored to 'Hello, '

History: Here are the saved states:
» 0: [2025-04-01 22:43:54] Hello, ...
  1: [2025-04-01 22:43:54] Hello, world!...
  2: [2025-04-01 22:43:54] Hello, world! 

## Benefits

* **Maintains Encapsulation**: Preserves object encapsulation while allowing state capture
* **Simplifies Originator**: Removes state saving/restoring responsibilities from the Originator
* **History Management**: Supports undo/redo functionality and state history tracking
* **Snapshot Isolation**: Provides isolated snapshots of state without affecting current operations
* **Clean Separation**: Separates state management from business logic